incremental loading data

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS scd;

CREATE TABLE IF NOT EXISTS scd.orders (
  order_id INT,
  customerid INT,
  customername STRING,
  customeremail STRING,
  productid INT,
  productname STRING,
  productcategory STRING,
  regionid INT,
  regionname STRING,
  country STRING,
  quantity INT,
  unitprice DECIMAL(10,2),
  totalamount DECIMAL(12,2)
);

INSERT INTO scd.orders VALUES
  (1, 101, 'Alice Smith', 'alice@example.com', 201, 'Laptop', 'Electronics', 301, 'North', 'USA', 2, 800.00, 1600.00),
  (2, 102, 'Bob Jones', 'bob@example.com', 202, 'Tablet', 'Electronics', 302, 'South', 'USA', 1, 300.00, 300.00),
  (3, 103, 'Carol Lee', 'carol@example.com', 203, 'Desk Chair', 'Furniture', 303, 'East', 'Canada', 4, 120.00, 480.00),
  (17, 101, 'Alice Smith', 'alice@example.com', 201, 'Laptop', 'Electronics', 301, 'North', 'USA', 2, 800.00, 1600.00),
  (4, 104, 'David Kim', 'david@example.com', 204, 'Monitor', 'Electronics', 304, 'West', 'USA', 3, 200.00, 600.00),
  (5, 105, 'Eva Green', 'eva@example.com', 205, 'Notebook', 'Stationery', 305, 'Central', 'UK', 10, 5.00, 50.00),
  (6, 106, 'Frank White', 'frank@example.com', 206, 'Pen', 'Stationery', 306, 'North', 'UK', 15, 2.00, 30.00),
  (7, 107, 'Grace Black', 'grace@example.com', 207, 'Headphones', 'Electronics', 307, 'South', 'Canada', 5, 50.00, 250.00),
  (16, 101, 'Alice Smith', 'alice@example.com', 201, 'Laptop', 'Electronics', 301, 'North', 'USA', 2, 800.00, 1600.00),
  (8, 108, 'Henry Brown', 'henry@example.com', 208, 'Mouse', 'Electronics', 308, 'East', 'USA', 7, 25.00, 175.00),
  (9, 109, 'Ivy Wilson', 'ivy@example.com', 209, 'Keyboard', 'Electronics', 309, 'West', 'Canada', 6, 45.00, 270.00),
  (10, 110, 'Jack Taylor', 'jack@example.com', 210, 'Desk Lamp', 'Furniture', 310, 'Central', 'USA', 3, 35.00, 105.00),
  (11, 111, 'Karen Young', 'karen@example.com', 211, 'Book', 'Stationery', 311, 'North', 'Canada', 12, 10.00, 120.00),
  (12, 112, 'Leo King', 'leo@example.com', 212, 'Printer', 'Electronics', 312, 'South', 'UK', 2, 150.00, 300.00),
  (13, 113, 'Mona Scott', 'mona@example.com', 213, 'Paper', 'Stationery', 313, 'East', 'UK', 20, 1.50, 30.00),
  (14, 114, 'Nina Adams', 'nina@example.com', 214, 'Desk', 'Furniture', 314, 'West', 'UK', 1, 250.00, 250.00),
  (15, 115, 'Oscar Reed', 'oscar@example.com', 215, 'Chair Mat', 'Furniture', 315, 'Central', 'Canada', 2, 40.00, 80.00);

In [0]:
%sql
ALTER TABLE scd.orders ADD COLUMNS (orderdate DATE);

UPDATE scd.orders
SET orderdate = CASE
  WHEN order_id = 1 THEN '2025-07-01'
  WHEN order_id = 2 THEN '2025-07-02'
  WHEN order_id = 3 THEN '2025-07-03'
  WHEN order_id = 4 THEN '2025-07-04'
  WHEN order_id = 5 THEN '2025-07-05'
  WHEN order_id = 6 THEN '2025-07-06'
  WHEN order_id = 7 THEN '2025-07-07'
  WHEN order_id = 8 THEN '2025-07-08'
  WHEN order_id = 9 THEN '2025-07-09'
  WHEN order_id = 10 THEN '2025-07-10'
  WHEN order_id = 11 THEN '2025-07-11'
  WHEN order_id = 12 THEN '2025-07-12'
  WHEN order_id = 13 THEN '2025-07-13'
  WHEN order_id = 14 THEN '2025-07-14'
  WHEN order_id = 15 THEN '2025-07-15'
  WHEN order_id = 16 THEN '2025-07-16'
  WHEN order_id = 17 THEN '2025-07-17'
  ELSE NULL
END;

In [0]:
%sql
SELECT * FROM workspace.scd.orders;

SCD-0: INSERT RECORD TO DIMENSION TABLE


In [0]:
%sql
CREATE TABLE IF NOT EXISTS scd.product_dim (
  productid int,
  productname string,
  productcategory string
) 

In [0]:
%sql
SELECT * FROM workspace.scd.orders;

In [0]:
%sql
INSERT INTO workspace.scd.product_dim 
SELECT productid, productname, productcategory 
FROM workspace.scd.orders;

In [0]:
%sql
select * from workspace.scd.product_dim;

SCD-1: UPSERT (UPDATE AND INSERT) USING MERGE

In [0]:
%sql
INSERT INTO workspace.scd.orders VALUES
(14, 114, 'Nina Adams', 'nina@example.com', 214, 'ManualDeSk', 'Furniture', 314, 'West', 'UK', 1, 250.00, 250.00,'2025-07-17'),
(15, 115, 'Oscar Reed', 'oscar@example.com', 230, 'Chair Mat', 'Furniture', 315, 'Central', 'Canada', 2, 40.00, 80.00,'2025-07-18');

In [0]:
%sql
CREATE OR REPLACE VIEW workspace.scd.view_product_dim
AS
SELECT distinct(productid) AS productid, productname, productcategory FROM workspace.scd.orders WHERE orderdate > '2025-07-16';

In [0]:
%sql
SELECT * FROM workspace.scd.view_product_dim;

In [0]:
%sql
MERGE INTO workspace.scd.product_dim AS trg
USING workspace.scd.view_product_dim AS src
ON trg.productid = src.productid
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *

In [0]:
%sql
SELECT * FROM workspace.scd.product_dim;

REAPETE